In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [15]:
# Örnek bir CNN modeli oluşturalım
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)  # Sınıf sayısı 2 olsun (örneğin, binary classification)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = x.view(-1, 32 * 56 * 56)  # Gerekirse boyutları yeniden ayarlayın
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [16]:
# Veri setini yükleyelim ve bölelim (örneğin, train, validation, test olarak)
def load_data():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    data_transform = datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Pneumonia/X-Ray', transform=transform)

    # Veri setini train, validation, test olarak bölecek şekilde ayarlayın
    train_size = int(0.75 * len(data_transform))  # %75 train
    test_size = len(data_transform) - train_size  # Geri kalan %25 test
    val_size = int(0.1 * train_size)  # %10 validation

    train_data, test_data = torch.utils.data.random_split(data_transform, [train_size, test_size])
    train_data, val_data = torch.utils.data.random_split(train_data, [train_size - val_size, val_size])

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)
    test_loader = DataLoader(test_data, batch_size=32)

    return train_loader, val_loader, test_loader


In [17]:
# CNN modelini eğitme ve değerlendirme fonksiyonu
def train_evaluate_cnn(model, train_loader, val_loader, test_loader, epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        # Eğitim aşaması
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Doğrulama aşaması
        model.eval()
        total_val, correct_val = 0, 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

        val_accuracy = 100 * correct_val / total_val
        print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%")

    # Test aşaması
    model.eval()
    total_test, correct_test = 0, 0
    true_labels, predicted_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    test_accuracy = 100 * correct_test / total_test
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    # Accuracy, Precision, Recall, F1 Score, Confusion Matrix, Classification Report
    print(f"Accuracy: {accuracy_score(true_labels, predicted_labels)}")
    print(f"Precision: {precision_score(true_labels, predicted_labels)}")
    print(f"Recall: {recall_score(true_labels, predicted_labels)}")
    print(f"F1 Score: {f1_score(true_labels, predicted_labels)}")
    print(f"Confusion Matrix:\n{confusion_matrix(true_labels, predicted_labels)}")
    print(f"Classification Report:\n{classification_report(true_labels, predicted_labels)}")


In [18]:
# Veri setini yükleyin ve eğitim ve değerlendirme fonksiyonunu çağırın
train_loader, val_loader, test_loader = load_data()
cnn_model = CNN()
train_evaluate_cnn(cnn_model, train_loader, val_loader, test_loader)

Epoch 1/5, Validation Accuracy: 82.51%
Epoch 2/5, Validation Accuracy: 94.88%
Epoch 3/5, Validation Accuracy: 98.35%
Epoch 4/5, Validation Accuracy: 98.35%
Epoch 5/5, Validation Accuracy: 98.35%
Test Accuracy: 98.14%
Accuracy: 0.9814448292924295
Precision: 0.9820742637644047
Recall: 0.9702719797596457
F1 Score: 0.9761374482978047
Confusion Matrix:
[[2433   28]
 [  47 1534]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      2461
           1       0.98      0.97      0.98      1581

    accuracy                           0.98      4042
   macro avg       0.98      0.98      0.98      4042
weighted avg       0.98      0.98      0.98      4042

